# Imports and Global Variables

In [1]:
from typing import List
from config.endpoints import MAINNET_BASE_URL
from helpers.utility import _make_get_request, _make_post_request
from helius import NFTAPI, NameAPI, BalancesAPI, WebhooksAPI, TransactionsAPI
from dotenv import load_dotenv
import requests
import pandas as pd
import numpy as np
import os
import json
from collections import defaultdict


In [2]:
# Persistent Variables

LAMPORT_SCALE = 1e9

In [3]:
load_dotenv()

True

In [4]:
# class TransactionsAPI:
#     def __init__(self, api_key: str):
#         self.base_url = MAINNET_BASE_URL
#         self.headers = {"Authorization": f"Bearer {api_key}"}
#         self.api_key_query = f"?api-key={api_key}"

#     def get_raw_transactions(
#         self, 
#         accounts: List[str], 
#         start_slot: int=None,
#         end_slot: int=None,
#         start_time: int=None,
#         end_time: int=None,
#         limit: int=None,
#         sort_order: str=None,
#         pagination_token: str=None
#         ):

#         path = "/v1/raw-transactions"
#         url = self.base_url + path + self.api_key_query
#         payload = {
#             "query": {
#                 "accounts": accounts,
#                 "startSlot": start_slot,
#                 "endSlot": end_slot,
#                 "startTime": start_time,
#                 "endTime": end_time
#             },
#             "options": {
#                 "limit": limit,
#                 "sortOrder": sort_order,
#                 "paginationToken": pagination_token
#             }
#         }
#         return _make_post_request(url, payload)

#     def get_parsed_transactions(self, transactions: List[str], commitment: str=None):
#         path = "/v0/transactions"
#         if commitment:
#             url = self.base_url + path + self.api_key_query + "?commitment=commitment"
#         else:
#             url = self.base_url + path + self.api_key_query
#         payload = {
#             "transactions": transactions
#         }
#         return _make_post_request(url, payload)

#     def get_parsed_transaction_history(
#         self, 
#         address: str, 
#         before: str='', 
#         until: str='', 
#         commitment: str='',
#         source: str='',
#         type: str=''
#         ):
        
#         path = f"/v0/addresses/{address}/transactions"
#         params = {
#             "before": before,
#             "until": until,
#             "commitment": commitment,
#             "source": source,
#             "type": type
#         }

#         url = self.base_url + path + self.api_key_query
#         print(f'url: {url}')
#         return _make_get_request(url, params=params)

In [5]:
HELIUS_API_KEY = os.getenv('HELIUM_API_KEY')

In [6]:
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'

# Flipside Data (Balances over time)

Here is the Flipside Query; when moving to live app will use the Flipside API to retrieve this dynamically: https://flipsidecrypto.xyz/studio/queries/f72285d7-f3d4-4596-a9cd-8785187005a2

In [7]:
os.getcwd()

'e:\\Projects\\solana_analysis_tool\\notebooks'

In [8]:
os.chdir('..')

In [9]:
balance_df = pd.read_csv('data/solana_balance_history_test.csv').dropna(how='all')

# Strip BOMs or invisible characters from column names
balance_df.columns = balance_df.columns.str.replace('\ufeff', '', regex=False).str.strip()

# Optionally: remove rows where any column has just a BOM or is empty/whitespace
balance_df = balance_df[~balance_df.apply(lambda row: row.astype(str).str.contains('\ufeff|^\s*$', regex=True)).any(axis=1)]

sol_mask = (
    (balance_df['MINT'] == 'So11111111111111111111111111111111111111111') &
    (balance_df['SYMBOL'].isna())
)

balance_df.loc[sol_mask, 'SYMBOL'] = 'SOL'
balance_df.loc[sol_mask, 'NAME'] = 'Solana'

In [ ]:
# Any nan values left should be LP tokens

balance_df[balance_df['SYMBOL'].isna()]

,BLOCK_TIMESTAMP,OWNER,MINT,PRE_BALANCE,BALANCE,TX_ID,SYMBOL,NAME
481,2024-01-27 20:02:00.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,21.284506,0.000000,AG9Wy4nnG7vgMgVwmBmiKUdfcGk9Sigh9XHBgUMq9LCfL8...,NaN,NaN
486,2024-01-27 20:00:42.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,21.284506,3kTQ7DYxvUoknsnHkHd4wMGCvXgq5CdXM6ZPUUnfJc1tGB...,NaN,NaN
494,2024-01-27 19:56:38.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,38.261386,0.000000,5XbLMj697ixEAzt6aZMD3fXx26cKYUwHmA2PmqMPYAcDry...,NaN,NaN
496,2024-01-27 19:54:37.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,38.261386,2JGMFWYbCcR51SgXUPejvvpRRHAs1xyxPgTz96DrrXNNmb...,NaN,NaN
506,2024-01-27 19:52:56.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,12.880316,0.000000,WMkk8qPz8xNk1kw1zaggErsBBbegDM6PRXWYiDvbDcU4Wv...,NaN,NaN
509,2024-01-27 19:51:51.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,10.000000,12.880316,t1EgxX8diQh5Ak4ptKnY6ajRu96dwE2R8yYJraQsSJ4Cxy...,NaN,NaN
511,2024-01-27 19:47:10.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,10.000000,DEYMhgGRURmjVATdnhMQAUsZFxHzRXftxPS3E5BGteS7VJ...,NaN,NaN
525,2024-01-12 00:29:26.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,B9uEZfxJoAsc51BZs6rNs8XtYVu5vfVrhbCuDCJ7mPEE,0.000000,5.000000,qWSnf1ioo2UCggYWbZDFa1KnAv8VUJgg7dKWGKAZFYP6q6...,NaN,NaN
526,2024-01-11 23:02:04.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,5.711749,0.000000,WNZ5j6dcGtjmigf8dXVbH72T4ks6Bo3kvtSgMBzL7trJdN...,NaN,NaN
528,2024-01-11 22:57:58.000,AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU,9HB4kAMLSYfGFfN142DKMyPyHyZQ8pXF8M1STbDudodY,0.000000,5.711749,5238f2M2JwJBSqSNb5k1L8B6mj5Z9D1XS9EEaKWzjKpZSf...,NaN,NaN


In [11]:
os.chdir('notebooks')

In [12]:
os.getcwd()

'e:\\Projects\\solana_analysis_tool\\notebooks'

# Helius API Calls

In [13]:
from helius import TransactionsAPI
transactions_api  = TransactionsAPI(HELIUS_API_KEY)

In [ ]:
use_cache = True

In [22]:
os.getcwd()

'e:\\Projects\\solana_analysis_tool\\notebooks'

In [23]:
os.chdir('..')

In [24]:
if not use_cache:
    parsed_transaction_history = transactions_api.get_parsed_transaction_history(address=test_address)

    with open('data/test_tx_history.json', 'w') as f:
        json.dump(parsed_transaction_history, f)
else:
    with open('data/test_tx_history.json') as f:
        parsed_transaction_history = json.load(f)

In [25]:
os.chdir('notebooks')

In [26]:
os.getcwd()

'e:\\Projects\\solana_analysis_tool\\notebooks'

In [27]:
print(json.dumps(parsed_transaction_history, indent=2))

[
  {
    "description": "5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr transferred a total 0.000005845 SOL to multiple accounts.",
    "type": "TRANSFER",
    "source": "SYSTEM_PROGRAM",
    "fee": 5825,
    "feePayer": "5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr",
    "signature": "6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxMTEx9RUZYt2SyFRvpPohpXm9RRABbVb2CEHgpdTzGv",
    "slot": 331670303,
    "timestamp": 1743941539,
    "tokenTransfers": [],
    "nativeTransfers": [
      {
        "fromUserAccount": "5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr",
        "toUserAccount": "B1rgyryCqZRazchK9fGifahLqn2LGBzkRByupHW7Dmu8",
        "amount": 1
      },
      {
        "fromUserAccount": "5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr",
        "toUserAccount": "ABXQf1Efhb3PRiJfB3dWs8mPgrHJmLrQBJZVKC1Y3YVj",
        "amount": 1
      },
      {
        "fromUserAccount": "5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr",
        "toUserAccount": "9WWzbnzzMBbyUHmPgL3v49D5awe6f6XDFNBLRJ6sZj

In [ ]:
len(parsed_transaction_history) # each tx is an array

98

In [ ]:
parsed_transaction_history[0].keys()

In [42]:
balance_df['TX_ID'] = balance_df['TX_ID'].str.lower()

In [ ]:
# Is slot the block number?

parsed_transaction_history[0].get('slot')

331670303

In [ ]:
# TX Id from Flipside dataset matches signatures in Helius dataset 

signatures = []

for ar in parsed_transaction_history:
    signature = ar['signature'].lower()
    if signature in balance_df['TX_ID'].unique():
        print(f'ar: {ar}')


ar: {'description': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr transferred a total 0.000005845 SOL to multiple accounts.', 'type': 'TRANSFER', 'source': 'SYSTEM_PROGRAM', 'fee': 5825, 'feePayer': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'signature': '6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxMTEx9RUZYt2SyFRvpPohpXm9RRABbVb2CEHgpdTzGv', 'slot': 331670303, 'timestamp': 1743941539, 'tokenTransfers': [], 'nativeTransfers': [{'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'B1rgyryCqZRazchK9fGifahLqn2LGBzkRByupHW7Dmu8', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'ABXQf1Efhb3PRiJfB3dWs8mPgrHJmLrQBJZVKC1Y3YVj', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': '9WWzbnzzMBbyUHmPgL3v49D5awe6f6XDFNBLRJ6sZj3s', 'amount': 1}, {'fromUserAccount': '5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr', 'toUserAccount': 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M

In [ ]:
#block_time (UTC)

pd.to_datetime(parsed_transaction_history[0].get('timestamp'), unit='s')

Timestamp('2025-04-06 12:12:19')

In [ ]:
# Account data

account_data = {}

for i in parsed_transaction_history[0].get('accountData'):
    accounts = i['account']
    if accounts and test_address in accounts:
        nativeBalanceChange = i['nativeBalanceChange']
        tokenBalanceChanges = i['tokenBalanceChanges']

        account_data['nativeBalanceChange'] = nativeBalanceChange / LAMPORT_SCALE
        account_data['tokenBalanceChanges'] = tokenBalanceChanges

account_data

{'nativeBalanceChange': 1, 'tokenBalanceChanges': []}

In [ ]:
#Tx type

parsed_transaction_history[0].get('type')

'TRANSFER'

In [ ]:
#Tx fee

parsed_transaction_history[0].get('fee') / LAMPORT_SCALE

5.825e-06

In [52]:
#Instruction data
instructions_data = {}

for i in parsed_transaction_history[0].get('instructions'):
    accounts = i['accounts']
    if accounts and test_address in accounts:
        data = i['data']
        program_id = i['programId']
        instructions_data['data'] = data
        instructions_data['programId'] = program_id

instructions_data

{'data': '3Bxs412MvVNQj175', 'programId': '11111111111111111111111111111111'}

In [37]:
def summarize_token_transfer(tx, wallet):
    net_token_flows = defaultdict(float)
    for t in tx.get("tokenTransfers", []):
        mint = t["mint"]
        raw_amount = t["rawTokenAmount"]["tokenAmount"]
        decimals = t["rawTokenAmount"]["decimals"]
        amount = int(raw_amount) / (10 ** decimals) if raw_amount.isdigit() else 0
        if t["fromUserAccount"] == wallet:
            net_token_flows[mint] -= amount
        elif t["toUserAccount"] == wallet:
            net_token_flows[mint] += amount
    return {
        "timestamp": tx["timestamp"],
        "signature": tx["signature"],
        "net_token_flows": dict(net_token_flows),
        "type": tx.get("type", "UNKNOWN"),
        "source": tx.get("source", "UNKNOWN")
    }


In [38]:
def summarize_native_transfer(tx, wallet):
    net_token_flows = defaultdict(float)
    for t in tx.get("nativeTransfers", []):
        amount = t.get('amount',0) / LAMPORT_SCALE
        if t["fromUserAccount"] == wallet:
            net_token_flows['SOL'] -= amount
        elif t["toUserAccount"] == wallet:
            net_token_flows['SOL'] += amount
    return {
        "timestamp": tx["timestamp"],
        "signature": tx["signature"],
        "net_token_flows": dict(net_token_flows),
        "type": tx.get("type", "UNKNOWN"),
        "source": tx.get("source", "UNKNOWN")
    }

In [39]:
native_tx_dict = summarize_native_transfer(parsed_transaction_history[0], test_address)
native_tx_dict

{'timestamp': 1743941539,
 'signature': '6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxMTEx9RUZYt2SyFRvpPohpXm9RRABbVb2CEHgpdTzGv',
 'net_token_flows': {'SOL': 1e-09},
 'type': 'TRANSFER',
 'source': 'SYSTEM_PROGRAM'}

In [40]:
token_tx_dict = summarize_token_transfer(parsed_transaction_history[0], test_address)
token_tx_dict

{'timestamp': 1743941539,
 'signature': '6mPqrKS4AKYJeZ3JLjjubHSwNB96oiJdFX1DgeZNZEBHxMTEx9RUZYt2SyFRvpPohpXm9RRABbVb2CEHgpdTzGv',
 'net_token_flows': {},
 'type': 'TRANSFER',
 'source': 'SYSTEM_PROGRAM'}

In [3]:
# helpers/utility.py
def _make_post_request(url: str, payload: dict, headers: dict) -> dict:
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError(f"Error: {response.status_code}: {response.text}")

In [4]:
# Initialize with your API key (ensure it's valid and active)
#HELIUS_API_KEY = os.getenv("api_key")  
tx_api = TransactionsAPI(api_key=HELIUS_API_KEY)

tx_api

In [ ]:
# # Example usage
# transactions = [""]  # Your transaction signatures
# response = tx_api.get_parsed_transactions(transactions)
# print(response)

TypeError: _make_post_request() missing 1 required positional argument: 'headers'

In [6]:
balances_api = BalancesAPI(HELIUS_API_KEY)
balances_api

In [15]:
balances_api.get_balances("hq8fC9VkxU6rv1QvMUkGKbhFR3MxSd7nCkCGaaXx9MA")

{'tokens': [{'tokenAccount': '2nKanNe6tDM5aXc2ZNvYeVKpdw4Y26dvzS19FmPiJJ4S',
   'mint': 'JD8gUc2qfvmwxtttRviHSRZ4pbGKy5nha2vdEULwx3r5',
   'amount': 3888000000,
   'decimals': 6},
  {'tokenAccount': 'GXnj7rSQtBUWcQbeP5QEyKJyv4br64uckb8xsS1sBY11',
   'mint': 'G4hdrdhnboNgFXo1uP21VwZ9zrNbxwZDmtpZt2Mnpump',
   'amount': 12126567051,
   'decimals': 6},
  {'tokenAccount': '4yLdi6X9xLRV3eezR3raibRoeAxotJA3n29uEsRfoNHL',
   'mint': 'Em9zr2tgSmGgRbz3kxyQeRXjRi9oc13wMu6cKam4zWFW',
   'amount': 74500000,
   'decimals': 6},
  {'tokenAccount': '6XWCWkwCdbBTzK5GrtTCVRQqXaB2QGuDHqZsXEt9KbBo',
   'mint': 'EWx3xDR76YfxF3VHwTLgwU9WYrwANw3yxX5vvk8Kpump',
   'amount': 119000000,
   'decimals': 6},
  {'tokenAccount': 'HSrx5V9PUuazeuen6ZYaid2o1KUbiLwDaRmJdjBmqrJZ',
   'mint': 'HD1sDvk9jjUgRv6XmVYje7ghV3udzCZhotgxSortBdga',
   'amount': 1434339522508,
   'decimals': 9},
  {'tokenAccount': '8kuTaQgk46Wo7PCuXfZuySEP85fZRgVbvCXkAfwVmt9S',
   'mint': 'Hk82ay38uGpYgqjdXXCiATavVQwAMaBNyxsUWbapmKuL',
   'amount': 